# PotentialNet モデルをダウンロード、ローカルでホスティング

このノートブックでは、SageMakerで学習させたPotentialNet Modelの使い方を紹介します。
まず、SageMaker Training Jobで作成された`model.tar.gz`ファイルをこのディレクトリにダウンロードしてください。そして、モデルファイルを以下のコマンドで解凍してください。すると、`best_test_model.pth`, `best_val_model.pth`, `model.pth`の3つのファイルが表示されます。

In [ ]:
!tar -zxvf model.tar.gz

`data_dir`をセットしてください。

In [ ]:
data_dir = 'graph_files_v2020_core_13_withPDBID'

次に、SageMaker Training Jobで行ったように、Custom Dataset / DataLoader オブジェクトを作成します。

In [ ]:
import sys
import os
from torch.utils.data import DataLoader, Dataset

sys.path.append('code/')

class MyDataset(Dataset):
    def __init__(self, lst_graph1_paths):
        super().__init__()
        
        self.len = len(lst_graph1_paths)
        self.lst = lst_graph1_paths
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        graphs1, label_dict = load_graphs(self.lst[index])
        graphs2, label_dict = load_graphs(self.lst[index].replace('_g1.bin', '_g2.bin'))
        label = label_dict['glabel']
        
        graphs1_batch = [dgl.batch([graphs1[i],graphs1[i+1]]) for i in range(0, int(len(graphs1)), 2)]
        bg = [tuple([graphs1_batch[i],graphs2[i]]) for i in range(0, int(len(graphs2)), 1)]
        
        return bg[0], label
    
def collate(data):
    graphs, labels = map(list, zip(*data))
    if (type(graphs[0]) == tuple):
        bg1 = dgl.batch([g[0] for g in graphs])
        bg2 = dgl.batch([g[1] for g in graphs])
        bg = (bg1, bg2) # return a tuple for PotentialNet
    else:
        bg = dgl.batch(graphs)
        for nty in bg.ntypes:
            bg.set_n_initializer(dgl.init.zero_initializer, ntype=nty)
        for ety in bg.canonical_etypes:
            bg.set_e_initializer(dgl.init.zero_initializer, etype=ety)

    labels = torch.stack(labels, dim=0)
    return bg, labels

In [ ]:
import glob
from dgl.data.utils import load_graphs
import dgl

lst_g1 = glob.glob(data_dir +"/"+ "**_g1.bin")
inf_data_set = MyDataset(lst_g1)

data_loader = DataLoader(
        dataset=inf_data_set,
        batch_size=1,
        shuffle=False,
        collate_fn=collate,
        pin_memory=True,
        num_workers=1,
    )

同じ構成のPotentialNetモデルを再作成し、学習済みモデルの重みをロードします。

In [ ]:
from dgllife.model import ACNN, PotentialNet
from configure import get_exp_configure
from utils import load_dataset, load_model, rand_hyperparams, set_random_seed
import torch

args = {}
args["model"] = "PotentialNet"
args["dataset_option"] = "PDBBind_refined_pocket_scaffold"
args["exp"] = "_".join([args["model"], args["dataset_option"]])

default_exp = get_exp_configure(args["exp"])
for i in default_exp.keys():
    args.setdefault(i, default_exp[i])
args['distance_bins'] =  [1.5, 2.5, 2.7, 2.9, 3.1, 3.3, 3.5, 4.5]

model = PotentialNet(n_etypes=(len(args['distance_bins'])+ 5),
                             f_in=args['f_in'],
                             f_bond=args['f_bond'],
                             f_spatial=args['f_spatial'],
                             f_gather=args['f_gather'],
                             n_rows_fc=args['n_rows_fc'],
                             n_bond_conv_steps=args['n_bond_conv_steps'],
                             n_spatial_conv_steps=args['n_spatial_conv_steps'],
                             dropouts=args['dropouts'])

In [ ]:
model.load_state_dict(torch.load('best_test_model.pth', map_location=torch.device('cpu')))
model.eval()

最後に、テストデータを用いて予測を実行します。

In [ ]:
for batch_id, batch_data in enumerate(data_loader):
        bg, labels = batch_data
        labels = labels
        bigraph_canonical, knn_graph = bg  # unpack stage1_graph, stage2_graph
        bigraph_canonical = bigraph_canonical
        knn_graph = knn_graph
        prediction = model(bigraph_canonical, knn_graph)
        print(prediction)